In [176]:
import clipboard, io

In [177]:
nouns = ("puppy", "car", "rabbit", "girl", "monkey")
verbs = ("runs", "hits", "jumps", "drives", "fucks") 
adv = ("crazily", "dutifully", "foolishly", "merrily", "occasionally")
adj = ("adorable", "clueless", "dirty", "odd", "stupid")

articles = ("A", "The")
pronouns = ("who", "that")


import random
def sentence(n):
    parts = []
#     parts.append(random.choice(articles))
#     parts.append(random.choice(nouns))
    for i in range(n):
        if not i % 2:
            part = " ".join((random.choice(articles), random.choice(adj), random.choice(nouns)))
        else:
            part = " ".join((random.choice(adv), random.choice(verbs)))
        parts.append(part)
    return " ".join(parts)

In [178]:
CR = 1
LF = 2
CRLF = 3
UNI = 4
EOLs = [
    "",
    "CR",
    "LF",
    "CRLF"
]
EOLs2 = {
    "CR": CR,
    "LF": LF,
    "CRLF": CRLF
}
class StringStorage():
    def __init__(self, eol, string, parts):
        self.eol = eol
        self.string = string
        self.parts = parts

In [218]:
def printr(s): print('"', s, '"', sep="")
    
class Strings():
    def __init__(self):
        self.strings = []
        self.maxn = 0
        
    def add(self, eol, s, frag):
        ss = StringStorage(eol, s, frag)
        self.strings.append(ss)
        if (len(frag) > self.maxn):
            self.maxn = len(frag)
    def __len__(self):
        return len(self.strings)
    
    def __iter__(self):
        return iter(self.strings)


def _readline_fix(line, eol):
    if line.endswith(eol):
        return line[:len(line)-len(eol)]
    return line
         
def _flatten(ob, l):
    for o in ob:
        if isinstance(o, (tuple, list)):
            _flatten(o, l)
        else:
            l.append(o)
    
def flatten(ob):
    ret = []
    _flatten(ob, ret)
    return ret
    
def generate(n):
    STRINGS = Strings()
    for i in range(2):
        slen = random.randint(1,3)
        eol = "\r", "\n", "\r\n"
        raw = sentence(slen)
        for sep in eol:
            s = raw.replace(" ", sep)
            for nl in "\r", "\n", "\r\n", None:
                b = io.StringIO(s, nl)
                if nl:
                    fragments = s.split(nl)
                else:
                    fragments = s.split("\r\n")
                    for i in range(len(fragments)):
                        fragments[i] = fragments[i].split("\n")
                    for i in range(len(fragments)):
                        for j in range(len(fragments[i])):
                            fragments[i][j] = fragments[i][j].split("\r")
                    fragments = flatten(fragments)
                STRINGS.add(nl, s, fragments)
    return STRINGS

In [219]:
tmpl = """const int MAXPARTS = %d;
struct StringStorage {
    EOL eol; int nparts;
    const char *string;
    const char *parts[MAXPARTS];
};

StringStorage strings[] = {
    %s
};"""

def crepr(s):
    return repr(s).replace("'", '"')


    

ssarr = []
STRINGS = generate(2)
for s in STRINGS:
    st = crepr(s.string)
    if s.eol == "\r":
        ieol = "CR"
    elif s.eol == '\n':
        ieol = "LF"
    elif s.eol == "\r\n":
        ieol = "CRLF"
    elif s.eol is None:
        ieol = "ANY"
    else:
        raise ValueError(s.eol)
    partsline = [crepr(p) for p in s.parts]
    for _ in range(STRINGS.maxn - len(s.parts)):
        partsline.append("0")
    parts = ", ".join(partsline)
    ssarr.append("{EOL::%s,%s %d, %s, {%s}}" % (ieol, " " * (4-len(ieol)), 
                                                len(s.parts), crepr(s.string), parts))

In [220]:
strings = ",\n    ".join(ssarr)
result = tmpl % (STRINGS.maxn, strings)
print(result)
clipboard.copy(result)

const int MAXPARTS = 3;
struct StringStorage {
    EOL eol; int nparts;
    const char *string;
    const char *parts[MAXPARTS];
};

StringStorage strings[] = {
    {EOL::CR,   3, "A\radorable\rpuppy", {"A", "adorable", "puppy"}},
    {EOL::LF,   1, "A\radorable\rpuppy", {"A\radorable\rpuppy", 0, 0}},
    {EOL::CRLF, 1, "A\radorable\rpuppy", {"A\radorable\rpuppy", 0, 0}},
    {EOL::ANY,  3, "A\radorable\rpuppy", {"A", "adorable", "puppy"}},
    {EOL::CR,   1, "A\nadorable\npuppy", {"A\nadorable\npuppy", 0, 0}},
    {EOL::LF,   3, "A\nadorable\npuppy", {"A", "adorable", "puppy"}},
    {EOL::CRLF, 1, "A\nadorable\npuppy", {"A\nadorable\npuppy", 0, 0}},
    {EOL::ANY,  3, "A\nadorable\npuppy", {"A", "adorable", "puppy"}},
    {EOL::CR,   3, "A\r\nadorable\r\npuppy", {"A", "\nadorable", "\npuppy"}},
    {EOL::LF,   3, "A\r\nadorable\r\npuppy", {"A\r", "adorable\r", "puppy"}},
    {EOL::CRLF, 3, "A\r\nadorable\r\npuppy", {"A", "adorable", "puppy"}},
    {EOL::ANY,  3, "A\r\nadorable\r\npupp

In [206]:
io.StringIO("A\r\nodd\r\nmonkey", "\r").readlines()

['A\r', '\r', 'odd\r', '\r', 'monkey']